In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv("./train.csv")
test_data = pd. read_csv("./test.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.isna().sum()

In [ ]:
def group_passengers():
    group = []

    for x in train_data["PassengerId"]:
        g = str(x).split("_")[0]
        group.append(g)
    return group

group = group_passengers()

train_data["Group"] = group

In [ ]:
train_data["Family"] = [str(x).split()[-1] for x in train_data["Name"].to_list()]

In [ ]:
train_data["Family"].isna = train_data.groupby("Group")["Family"].agg(lambda x: x.value_counts().index[0])

train_data.head()

In [ ]:
train_data.to_csv("./check.csv")

In [ ]:
#train_data["Cabin"].isna = train_data.groupby("Family")["Cabin"].agg(lambda x: x.value_counts().index[0])

In [ ]:
def split_cabin_info(cabin: str):
    if pd.isna(cabin):
        return np.nan, np.nan, np.nan

    x = cabin.split("/")

    return x[0], int(x[1]), x[2]

In [ ]:
deck = []
num = []
shipSide = []

for x in train_data["Cabin"]:
    d, n, s = split_cabin_info(x)
    deck.append(d)
    num.append(n)
    shipSide.append(s)

In [ ]:
train_data["Deck"], train_data["Num"], train_data["shipSide"] = deck, num, shipSide

In [ ]:
train_data.head()

In [ ]:
train_data.groupby(["shipSide","Deck",])["Transported"].sum("Transported").plot(subplots=True, kind="bar")

In [ ]:
train_data["NumBin"] = pd.cut(train_data["Num"], bins=10)

train_data.groupby(["NumBin"])["Transported"].sum("Transported").plot(kind="bar")

In [ ]:
train_data.query("CryoSleep == True")["Transported"].value_counts().plot(kind="bar")

In [ ]:
train_data.head()

In [ ]:
#AgeGroup

def return_age_group():
    pass

In [ ]:
#HomePlanet and Destination via family and group

In [ ]:
#totalSpend via summing all spending

In [ ]:
#Set spending of all people in Cryo to zero
train_data.loc[train_data["CryoSleep"] == True ,["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]] = 0.0

In [ ]:
#Create column for totalSpent on all amnenities
train_data["totalSpent"] = train_data[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)

In [ ]:
#Set age 12 to all awake people with no spending
train_data["Age"] = np.where((train_data.CryoSleep == False) & (
    train_data.Age.isna()) & (train_data.totalSpent == 0), 4, train_data.Age)


In [ ]:
train_data.groupby("VIP")["totalSpent"].mean("totalSpent")

In [ ]:
train_data.VIP.value_counts()

In [ ]:
#impute VIP status by average spending
train_data["VIP"] = np.where((train_data["VIP"].isna() & train_data["totalSpent"] > 4425 * 0.75), True, False)

In [ ]:
train_data.loc[(train_data.CryoSleep == False) & (train_data.Age.isna()) & (train_data.totalSpent == 0)][["Age", "totalSpent", "CryoSleep"]].sort_values(by="totalSpent")

In [ ]:
imp_data = train_data.dropna()
imp_data

In [ ]:
training_cols = ["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Deck", "shipSide", "NumBin", "totalSpent"]

In [ ]:
#train_data.dropna(inplace=True)

In [ ]:
X = train_data[training_cols]

X.shape

In [ ]:
y = train_data["Transported"]

In [ ]:
X.dropna()

In [ ]:
sns.catplot(data=train_data, y="totalSpent",kind="violin")

In [ ]:
sns.catplot(data=train_data[["Age", "Transported"]], kind="violin", y="Age")

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "totalSpent"]
cat_cols = ["HomePlanet", "Destination", "VIP", "Deck", "shipSide", "NumBin"]

In [ ]:
ct_X = ColumnTransformer(
    transformers=[
       ("impute_num", IterativeImputer(), num_cols),
       ("scale", RobustScaler(), num_cols),

       ("impute_cat", IterativeImputer(), cat_cols),
       ("encode", OneHotEncoder(), cat_cols)
    ], 
    remainder="passthrough"
)

In [ ]:
X = ct_X.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.svm import SVC

s = SVC().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

s.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

b = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5,
                      max_features=0.5).fit(X_train, y_train)


In [ ]:
b.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

r = RandomForestClassifier(n_estimators=20).fit(X_train, y_train)

In [ ]:
r.score(X_test, y_test)